# Assignment 4 - Graph Convolutional Networks
## Deep Learning Course - Vrije Universiteit Amsterdam, 2021

#### Instructions on how to use this notebook:

This notebook is hosted on Google Colab. To be able to work on it, you have to create your own copy. Go to *File* and select *Save a copy in Drive*.

You can also avoid using Colab entirely, and download the notebook to run it on your own machine. If you choose this, go to *File* and select *Download .ipynb*.

The advantage of using Colab is that you can use a GPU. You can complete this assignment with a CPU, but it will take a bit longer. Furthermore, we encourage you to train using the GPU not only for faster training, but also to get experience with this setting. This includes moving models and tensors to the GPU and back. This experience is vary valuable because for many interesting models and large datasets (like large CNNs for ImageNet, or Transformer models trained on Wikipedia), training on GPU is the only feasible way.

The default Colab runtime does not have a GPU. To change this, go to *Runtime - Change runtime type*, and select *GPU* as the hardware accelerator. The GPU that you get changes according to what resources are available at the time, and its memory can go from a 5GB, to around 18GB if you are lucky. If you are curious, you can run the following in a code cell to check:

```sh
!nvidia-smi
```

Note that despite the name, Google Colab does  not support collaborative work without issues. When two or more people edit the notebook concurrently, only one version will be saved. You can choose to do group programming with one person sharing the screen with the others, or make multiple copies of the notebook to work concurrently.

**Submission:** Upload your notebook in .ipynb format to Canvas. The code and answers to the questions in the notebook are sufficient, no separate report is expected. 

In [ ]:
!nvidia-smi

Sun Dec 19 20:33:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    75W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Introduction

Graphs are very useful data structures that allow us to represent sets of entities and the way they are related among each other. In a graph, entities are also known as *nodes*, and any link between entities is also called an *edge*.

Examples of real world objects that can be modeled as graphs are social networks, where entities are people and relations denote friendship; and molecules, where entities are atoms and relations indicate a bond between them.

There has been increased interest in the recent years in the application of deep learning architectures to graph-structured data, for tasks like predicting missing relations between entities, classifying entities, and classifying graphs. This interest has been spurred by the introduction of Graph Convolutional Networks (GCNs).

In this assignment, you will implement and experiment with one of the first versions of the GCN, proposed by Thomas Kipf and Max Welling in their 2017 paper, [Semi-supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907). In particular, the goals of this assignment are to

- Understand how GCNs are formulated
- Implement the GCN using PyTorch
- Train and evaluate a model for semi-supervised node classification in citation networks
- Train and evaluate a model for binary classification of molecules

### Representing graphs

Suppose we have the following graph:

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/01-graph.png" width="200">

This is an undirected graph (since the edges have no specified direction) with 4 nodes. One way to represent the connectivity structure of the graph is by means of the **adjacency matrix**. The $i$-th row of the matrix contains a 1 in the $j$-th column, if nodes $i$ and $j$ are connected. For an undirected graph like the one above, this means that the adjacency matrix

- Is symmetric (e.g. an edge between 0 and 2 is equivalent as an edge between 2 and 0)
- Is square, of size $n\times n$ where $n$ is the number of nodes

The adjacency matrix for the graph above is then the following:

$$
A =
\begin{bmatrix}
0 & 0 & 1 & 0 \\ 
0 & 0 & 1 & 0 \\
1 & 1 & 0 & 1 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$

A second matrix of interest is the **degree matrix**. This is a diagonal matrix where the $i$-th element of the diagonal indicates the number of edges connected to node $i$. Note that these can be obtained from $A$ by summing across the columns, or the rows. For our example, the degree matrix is

$$
D = \begin{bmatrix}
1 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 \\
0 & 0 & 3 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

For specific applications, each node in the graph will have an associated vector of features $x\in\mathbb{R}^c$. If our graph is a social network, then the vector of features can contain information like age, location, and musical tastes, in a specific numeric format. In the case of a molecule, the node could represent an atom and have features like the atomic mass, etc. We can lay out the features in a matrix $X\in\mathbb{R}^{n\times c}$, so that the feature vector for node $i$ is in the $i$-th row.

### Loading a citation network

To move to a real world example, we will start with the Cora dataset. This dataset represents a citation network, where nodes are scientific publications, edges denote citations between them, and features are a [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) extracted from their contents.

This graph contains labels for nodes, that represent a specific topic. We will use these for a node classification task.

To easily load it, we will use [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/index.html) (PyG), a deep learning library for graph-structured data.

In [ ]:
# Install PyTorch Geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu113.html


We can now use the library to download and import the dataset. Initializing the `Planetoid` class returns a `Dataset` object that can contain multiple graphs. In this task we will only use the `Cora` dataset (the citation network) and hence, we will select only the first element.

In [ ]:
from torch_geometric.datasets import Planetoid
data = Planetoid(root='data/Planetoid', name='Cora')[0]

In [ ]:
# Packages
import numpy as np
import torch

from scipy.sparse import coo_matrix
from scipy.sparse import diags
from scipy.sparse import block_diag

from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
from torch_scatter import scatter_max

from torch_geometric.nn import global_mean_pool
from torch_geometric.data import DataLoader




#### Question 1 (0.25 pt)

The `data` object is an instance of the `Data` class in PyG. Check the [documentation](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html) and report the following properties of the graph:

- Number of nodes
- Number of edges 
- The dimension $c$ of the feature vectors $x\in\mathbb{R}^c$
- The number of targets for the classification task

In [ ]:
# Your answer here
print("nodes: ", data.num_nodes)
print("edges: ", data.num_edges)
print("dimension c of features", data.num_node_features)

dataset = Planetoid(root='data/Planetoid', name='Cora')
print("number of classes: ",dataset.num_classes)


nodes:  2708
edges:  10556
dimension c of features 1433
number of classes:  7


#### Question 2 (0.25 pt)

In PyG, edges are provided in a tensor of shape (2, number of edges). You can access it via `data.edge_index`. Each column in this tensor contains the IDs for two nodes that are connected in the graph.

We saw that in an undirected graph, an edge between nodes $i$ and $j$ adds a value of 1 to positions $(i, j)$ and $(j, i)$ of the adjacency matrix. Is this also true for the edge index? That is, if there is an edge $(i, j)$ in `data.edge_index`, is there also an edge for $(j, i)$? This is important to know for the next steps of the implementation.

In [ ]:
# Your answer here

#check if the graph is undirected 
print("The graph is undirecred: ",data.is_undirected())

#check if  there is an edge  (i,j)  in data.edge_index, is there also an edge for  (j,i) ?
def check_directed():
  row_1 = np.array(data.edge_index[0])
  row_2 = np.array(data.edge_index[1])
  for i in range(len(row_1)):
    check = False
    for j in range(len(row_2)):
      if row_1[i] == row_2[j] and row_2[i] == row_1[j]:
        check = True
        break
    if not check:
      print("There is an edge (i,j) in data.edge_index, but edge  (j,i)  in data.edge_index is NOT found")
      return
  print("There is an edge  (i,j)  in data.edge_index and edge (j,i) is also in data.edge_index")


check_directed()


The graph is undirecred:  True
There is an edge  (i,j)  in data.edge_index and edge (j,i) is also in data.edge_index


#### Question 3 (0.5 pt)

In graphs, especially large ones, the adjacency matrix is **sparse**: most entries are zero. Sparse matrices allow for efficient storage and computation.

To prepare and pre-process sparse matrices, we will use [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html). Once the matrices are ready, we will convert them to PyTorch tensors.

We will use the [Sparse COO format](https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)). We encourage you to first get familiar with how it works after continuing with the assignment.

- Use the [`scipy.sparse.coo_matrix()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) function to build the adjacency matrix. Think of what arguments are needed, and how you can obtain them from the graph data loaded above.
- Use the `sum()` method of sparse matrices, together with `scipy.sparse.diags()`, to compute the degree matrix using the definition above.

Both resulting matrices must be sparse of type `float32`.

In [ ]:
# Your answer here
def data_prep(data):
  row = np.array(data.edge_index[0])
  col = np.array(data.edge_index[1])
  d = np.ones(len(data.edge_index[0]), dtype=np.float32)

  A = coo_matrix((d, (row, col)), shape=(data.num_nodes, data.num_nodes))
  A.toarray()

  D = diags(np.array(A.sum(axis=0))[0], 0)

  return A, D

A, D = data_prep(data)

You might wonder why we suggest to use a scipy sparse matrix, while also PyTorch supports them. The reason is that in the next step, we will be multiplying two sparse matrices, an operation not supported in PyTorch. PyTorch only allows multiplying a sparse matrix with a dense one, something which we will be doing at a later stage.



### The Graph Convolutional Network

The goal of the graph convolution is to take the feature vectors of all nodes $X\in\mathbb{R}^{n\times c}$, and propagate them along the existing edges, to obtain updated representations $Z\in\mathbb{R}^{n\times d}$.


The GCN is initially motivated as performing a convolution, similarly as it is done in CNNs for images, for graph-structured data. In Kipf and Welling (2017), a theoretical derivation leads to the following formula:

$$
Z = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}XW
$$

where

- $W\in\mathbb{R}^{c\times d}$ is a matrix of parameters to be learned via gradient descent
- $\tilde{A} = A + I_n$, where $I_n$ is an $n\times n$ identity matrix
- $\tilde{D}$ is the degree matrix computed with $\tilde{A}$ as the adjacency matrix

If we define $\hat{A} = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}$, the graph convolution can be written as $Z = \hat{A}XW$.

#### Question 4 (0.25 pt)

Given the formula for the GCN, explain why it operates by propagating feature vectors across the graph. To answer this, it might be useful to recall the definitions of the adjacency and degree matrices, and how they are involved in the formula.

# Answer Q4
The adjacency matrix is a square matrix of size $n \times n$ used to represent a graph. The elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph. The matrix is symmetric if the edges are undirected. adding $I_n$, will give a small change, because now the Adjendancy matrix also place's a one at all edges (i,i). This could be interpreted as self connections.

The degree matrix of a graph is a diagonal matrix. This matrix contains information about the degree of each vertex that is, the number of edges attached to each vertex $\tilde{D}$ is equal to $D + I_n$. Taking $\tilde{D}^{-1/2}$ is an operation, which can be (due to diagonal matrix properties) be performed element wise on every element on the diagonal. $X\in\mathbb{R}^{n\times c}$ is the matrix, which summarices the features (c) of every node (n). 

Before a graph based regularization was used, which forces nodes that are close together to be labelled the same. This has as downside that it might restrict modelling capacity as the edges does not neccessarily have to encode similarities. Those similarities could also be encode in for example futures. 

Therefor we use backpropogation of the features, where first all elements in $\tilde{A}$ are normalized using by $\frac{1}{\sqrt{d_i d_j}}$, this leads for a larger decrease for edges of the matrix A, when the nodes have a lot of connections, compared to when the nodes only have a few connections. Next we want to use additional information, which are represented in the features. If we would not backpropogate this information it would not be included in the model, therefor we want to backpropogate the feature values, using this normalization term. So that node x includes some portion of the information of their neighbours. Therefor also the $I_n$ was added, because otherwise the features for the node itself would not be included in the learning.






#### Question 5 (0.5 pt)

Compute the **normalized adjacency matrix** $\hat{A}$. The result should be a sparse matrix.

In [ ]:
# Your answer here
def normalized_adjacency(A, D):
  A_t = A + diags(np.ones(A.shape[0]) - A.diagonal())
  D_t = diags(np.array(A_t.sum(axis=0))[0], 0)

  D_t_power = D_t.power(-0.5)
  D_t_power.toarray()

  A_norm = coo_matrix(A_t.dot(D_t_power).transpose().dot(D_t_power))

  return A_norm

A_norm = normalized_adjacency(A, D)

#### Question 6 (0.5 pt)

So far we have used scipy to build and compute sparse matrices. Since we want to train a GCN with PyTorch, we need to convert $\hat{A}$ into a sparse PyTorch tensor. You can do this with the [`torch.sparse_coo_tensor()`](https://pytorch.org/docs/stable/generated/torch.sparse_coo_tensor.html) function, making sure to specify `torch.float` as the type.

In [ ]:
# Your answer here
def to_torch(A_norm):
  values = A_norm.data
  indices = np.vstack((A_norm.row, A_norm.col))

  i = torch.LongTensor(indices)
  v = torch.FloatTensor(values)
  shape = A_norm.shape

  A_hat = torch.sparse_coo_tensor(i, v, torch.Size(shape), dtype=torch.float32)

  return A_hat
A_hat = to_torch(A_norm)

In [ ]:
A_hat

tensor(indices=tensor([[   0,  633, 1862,  ..., 1473, 2706, 2707],
                       [   0,    0,    0,  ..., 2707, 2707, 2707]]),
       values=tensor([0.2500, 0.2500, 0.2236,  ..., 0.2000, 0.2000, 0.2000]),
       size=(2708, 2708), nnz=13264, layout=torch.sparse_coo)

#### Question 7 (0.5 pt)

We now have all the ingredients to build a GCN layer. Implement a class (inheriting from `torch.nn.Module`) with a learnable matrix of weights $W\in\mathbb{R}^{c\times d}$. Make sure to

- Call this class `GCNLayer`
- The `__init__()` constructor should take as argument the number of input and output features.
- Use `torch.nn.init.kaiming_uniform_` to initialize $W$.
- Define the `forward` method, which takes as input $X$ and $\hat{A}$ and returns $Z$. Note that multiplications involving the sparse matrix $\hat{A}$ have to be done with `torch.spmm`. 

Once you have implemented the class, instantiate a layer with the correct number of input features for the Cora dataset, and a number of output features of your choice. Do a forward pass and report the shape of the output tensor.

In [ ]:
# Your answer here
class GCNLayer(nn.Module):
  """
  Inits a weight matrix
  Inputs:
    c_in - Number of features
    c_out - Number of channels it wants as output
  """
  def __init__(self, c_in, c_out):
      
      super(GCNLayer, self).__init__()
      self.weight = Parameter(torch.Tensor(c_in, c_out))
      torch.nn.init.kaiming_uniform_(self.weight) 

  def forward(self, X, A_hat):
      """
      Inputs:
          X - Tensor of shape [#nodes, c_in] or [#nodes, c_in]
          A_hat - Adjacency matrices of the graph. 
      """
      Z = torch.matmul(torch.sparse.mm(A_hat, X), self.weight)

      return Z

# Init class
layer = GCNLayer(c_in=data.num_node_features, c_out=dataset.num_classes)

# Calculate prediction for Z
layer(data.x, A_hat)

tensor([[ 1.4676, -0.4318,  0.1197,  ...,  0.0060,  1.4398, -0.2111],
        [ 0.1573, -0.6981,  0.5620,  ...,  1.4054,  0.7447, -0.3761],
        [ 0.6501, -2.1099, -1.6343,  ...,  0.9926, -1.3912,  0.3917],
        ...,
        [ 0.4324, -1.5535, -0.5636,  ..., -2.2149,  0.6109, -0.3303],
        [ 1.3482, -2.7836,  0.8372,  ..., -0.3628,  1.1693, -1.5509],
        [ 1.5006, -1.8595,  0.5263,  ...,  0.1768,  0.8470, -0.6499]],
       grad_fn=<MmBackward0>)

#### Question 8 (0.5 pt)

As we have seen so far, the GCN layer implements a special type of linear transformation of the inputs. However, it is often beneficial in deep learning to stack multiple, non-linear transformations of the input features. Implement a second module class for a model with two GCN layers (use the module you implemented in the previous question).

- Call this class `GCN`
- The constructor must now take as input the number of input features, the output dimension of the first layer (this is the hidden layer), and the output dimension of the output layer.
- In the forward pass, add a ReLU activation function after the first layer.

In [ ]:
# Your answer here
class GCN(nn.Module):
  """
  Inits a weight matrix
  Inputs:
    c_in - Number of features
    c_hidden - Number of channels in hidden layer
    c_out - Number of channels it wants as output
  """
  def __init__(self, c_in, c_hidden, c_out):
      super(GCN, self).__init__()
      self.layer1 = GCNLayer(c_in, c_hidden)
      self.layer2 = GCNLayer(c_hidden, c_out)

  def forward(self, X, A_hat):
    """
    Performs one GCN pass with relu activation and then one more GCN forward pass
    Inputs:
        X - Tensor of shape [#nodes, c_in]
        A_hat - Adjacency matrices of the graph. 
    """

    x = F.relu(self.layer1(X, A_hat))
    Z = self.layer2(x, A_hat)

    return Z
    
model_Q8 = GCN(c_in=data.num_node_features,c_hidden=20, c_out=dataset.num_classes)
model_Q8(data.x, A_hat).shape

torch.Size([2708, 7])

### GCNs for semi-supervised node classification

Now that we have a GCN with two layers, we can test its performance in a node classification task. We will pass the input node features $X$ through the GCN layers, and the output will be of size $n\times k$ where $k$ is the number of classes (which you found in question 1). The label denotes the topic an article in the citation network belongs to (e.g. physics, computer science, etc.).

#### Question 9 (1.5 pt)

Note that the `data` object contains all labels (for all splits) in `data.y`, and binary masks for the train, validation, and test splits in `data.train_mask`, `data.val_mask`, and `data.test_mask`, respectively. These masks are the same size as `data.y`, and indicate which labels belong to which split.

- Create a GCN with two layers (using the class from the previous question), with 32 as the hidden dimension, and the number of output features equal to the number of classes in the Cora dataset.

- Use the Adam optimizer with a learning rate of 0.01.

- Implement a training loop for the GCN. At each step, pass $X$ and $\hat{A}$ to the GCN to obtain the logits. Compute the mean cross-entropy loss **only for the training instances**, using the binary masks.

- After each training step, evaluate the accuracy for the validation instances.

- Train for 100 epochs. Once training is finished, plot the training loss and validation accuracy (in a graph in function of the epoch number), and report the accuracy in the test set.

You should obtain an accuracy over 75% on both the validation and test sets. You can also compare your results with the original paper, which also contains results for the Cora dataset. Give a brief discussion on the results of your experiments.

Note that in contrast with other tasks, like image classification on some datasets, we don't use mini-batches here. The whole matrix of features and the adjacency is passed to the GCN in one step.

In [ ]:
# Your answer here
def forward(data, model, loss_cross, optim, A_hat, mask, mode="train"):
  # Compute the model prediction
  pred = model(data.x, A_hat)[mask]
  #compute Loss & accuracy  
  loss = loss_cross(pred, data.y[mask])
  winners = pred.argmax(dim=1)
  corrects = (winners == data.y[mask])
  accuracy = corrects.sum().float() / mask.sum()

  # Training step
  if mode == "train":
    optim.zero_grad()
    loss.backward()
    optim.step()

  return loss, accuracy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(c_in=data.num_node_features,c_hidden=32, c_out=dataset.num_classes)
model.to(device)
data = dataset[0].to(device)
A_hat = A_hat.to(device)

learning_rate = 0.01
batch_size = 1
epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=5e-4)
model.train()

loss_cross = nn.CrossEntropyLoss()


for e in range(epochs):
  loss, accuracy = forward(data, model, loss_cross, optimizer, A_hat, data.train_mask)
  print("Epoch number", e+1, "Loss:", loss.item(), "Acc", accuracy.item())

print("---Validation---")
loss, accuracy = forward(data, model, loss_cross, optimizer, A_hat, data.test_mask, mode="test")
print("Loss:", loss.item(), "Acc", accuracy.item())

print("---Test---")
loss, accuracy = forward(data, model, loss_cross, optimizer, A_hat, data.val_mask, mode="validation")
print("Loss:", loss.item(), "Acc", accuracy.item())

Epoch number 1 Loss: 2.5955398082733154 Acc 0.1071428582072258
Epoch number 2 Loss: 1.9912554025650024 Acc 0.18571428954601288
Epoch number 3 Loss: 1.6056880950927734 Acc 0.3642857074737549
Epoch number 4 Loss: 1.335476279258728 Acc 0.5928571224212646
Epoch number 5 Loss: 1.1130262613296509 Acc 0.699999988079071
Epoch number 6 Loss: 0.9195192456245422 Acc 0.8071428537368774
Epoch number 7 Loss: 0.7525983452796936 Acc 0.8714285492897034
Epoch number 8 Loss: 0.6122068166732788 Acc 0.9142857193946838
Epoch number 9 Loss: 0.49662983417510986 Acc 0.9357143044471741
Epoch number 10 Loss: 0.4024525284767151 Acc 0.9571428298950195
Epoch number 11 Loss: 0.3264150023460388 Acc 0.9642857313156128
Epoch number 12 Loss: 0.26531684398651123 Acc 0.9785714149475098
Epoch number 13 Loss: 0.21611523628234863 Acc 0.9857142567634583
Epoch number 14 Loss: 0.17680610716342926 Acc 0.9928571581840515
Epoch number 15 Loss: 0.1452232003211975 Acc 0.9928571581840515
Epoch number 16 Loss: 0.11918964236974716 Acc 

#### Question 10 (0.5 pt)

The paper introduces GCNs as a way to solve a *semi-supervised* classification problem.

- What makes this problem semi-supervised?
- What is the proportion of labeled data used for training with respect to labeled data in the validation and test sets? What is difference in this context with other benchmark tasks in machine learning, like image classification with MNIST?
- Why do you think the GCN performs well in this semi-supervised scenario?

# Q10 ANSWER
We speak about semi-supervised learning if the data is only partially labelled. In this specific example "data.y" includes all labels, but we don't use all labels. The model is fed with a complete graph of 2708 nodes, then we use only a few labels to make a prediction and back propogate the loss through the complete network 

We train the model on the complete data set with 2708 nodes, of these 2708 nodes we use only 5% of the data labels, this amount is really small and shows the power of the model as labelling data is a time expensive job. In other supervised benchmark task a lot more labelled data is fed to the machines as it increases understanding, there the goal is to maximize the amount of train data in such a way data the test and validation set are accurate enough to test the and tweek the model performances. But why this difference? A supervised model is trained to do prediction on new data let's say some unseen letteer in MNIST case, but the GCN's are build to predict labels in a network, if you build a network and you already feed it all the labels, why the go through all the trouble?

GCN's are especially powerfull in semi-supervised settings, because it understands relation between labelled and unlabelled data. Where image classification only uses an input and weights, the GCNs have next to the input and weights knowledge about how neurons are connected. This leads to a better understanding of also unlabelled data.


### Loading a dataset of proteins

In the previous sections you learned how to pass the adjacency matrix of a graph with a couple of thousand of nodes, to classify each node with a particular label. A different and useful application of GCNs is graph classification.

In contrast with the previous part, where there was a single, big graph, in graph classification we have multiple graphs, and each graph can be assigned a label. In this part of the assignment you will implement a classifier for proteins.

[Proteins](https://en.wikipedia.org/wiki/Protein_(nutrient)) are parts of the buildings block of life. They consist of chains of amino acids, and can take many shapes. In the PROTEINS dataset, proteins are represented as graphs, where the nodes are amino acids, and an edge between them indicates that they are 6 [Angstroms](https://en.wikipedia.org/wiki/Angstrom) apart. All graphs have a binary label, where 1 means that the protein is not an enzyme.

We will start by loading and examining this dataset.

In [ ]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='data/TU', name='PROTEINS', use_node_attr=True)

#### Question 11 (0.25 pt)

Unlike in the previous part, where we selected the first element returned by the loading function, note that here we get all the elements returned by `TUDataset()`. `dataset` is an interable object, that has some similar behaviors as a Python list: you can call `len()` on it, and you can takes slices from it.

Each element in `dataset` is a `Data` object containing a graph that represents a protein. This is the same type of object that we used in the previous part to store the Cora citation network.

Knowing this, answer the following:

- How many proteins (graphs) are there in `dataset`?
- Take any protein from `dataset`. How many nodes and edges does it contain? What is its label? How many features does each node have?

In [ ]:
# Your answer here
print("number of proteins (graphs) in dataset: ", len(dataset))


data = dataset[0]

print("number of nodes of a protein: ", data.num_nodes)
print("number of edges of a protein: ", data.num_edges)
print("number of features of each node in a protein: ", data.num_node_features)
print("labels: ", data.y)



number of proteins (graphs) in dataset:  1113
number of nodes of a protein:  42
number of edges of a protein:  162
number of features of each node in a protein:  4
labels:  tensor([0])


#### Question 12 (0.5 pt)

To properly train and evaluate our model, we need training, validation, and test splits.

For reproducibility purposes, we generate a random tensor of indices for you. Use it to extract the three splits from `dataset`.

For training, take 80% of the indices (starting from the first element in `indices`), then the following 10% for validation, and the remaining 10% for testing. You can use the indices to index `dataset`.

Call the resulting splits `train_dataset`, `valid_dataset`, and `test_dataset`.

In [ ]:
# Don't erase the following three lines
torch.random.manual_seed(0)
indices = torch.randperm(len(dataset))

# Your answer here
train_dataset = []
valid_dataset = []
test_dataset = []
for i in range(len(indices)):
  if i <= .8*len(dataset):
    train_dataset.append(dataset[indices[i]])
  elif i <= .9*len(dataset):
    valid_dataset.append(dataset[indices[i]])
  else:
    test_dataset.append(dataset[indices[i]])

### Working with a batch of graphs

When working with the Cora dataset, you used the information in `data.edge_index` to build the sparse normalized adjacency matrix $\hat{A}$ that is required by the GCN. We could do something similar here: for each graph, we build $\hat{A}$, and pass it to the GCN. However, if the number of graphs is big, this can really slow down training.

To avoid this, we will resort to a very useful trick that also allows us to reuse the same GCN you implemented previously. The trick makes it possible to do a forward pass through the GCN for multiple, disconnected graphs at the same time (instead of only one), much like when you train with mini-batches for other kinds of data.

Let's first revisit the propagation rule of the GCN, $Z = \hat{A}XW$, with an illustration (we have omitted the cells of $X$ and $W$ for clarity):

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/02-gcn-forward.png">

If we have multiple graphs, we can still use the same propagation rule, if we

- Set $\hat{A}$ to be a block diagonal matrix, where the blocks are the different adjacency matrices of the graphs
- Concatenate the feature matrices along the first dimension

This is illustrated in the following figure, for a batch of 3 graphs. Note that the elements outside of the blocks are zero.

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/02-gcn-batch-forward.png">

The resulting adjacency matrix $\hat{A}_B$ can also be built as a sparse matrix, and once we have it together with the concatenated matrix of features, the computation of the graph convolution is exactly the same as before. Note how this trick also allows us to process graphs with different sizes and structures.


#### Question 13 (0.5 pt)


Just as the citation network, the graphs in each of the datasets you created in Question 12 also have an `edge_index` attribute, which can be used to compute the normalized adjacency matrix $\hat{A}$, for each graph.

Reusing your code for Questions 3 and 5, define a function `get_a_norm()` that takes as input an element of a dataset (e.g. `train_dataset[0]`), and returns a `scipy.sparse` matrix containing $\hat{A}$.

Note that an element of a dataset has properties like `num_edges`, `num_nodes`, etc. which you can use here.

In [ ]:
# Your answer here
def get_a_norm(data):
  row = np.array(data.edge_index[0])
  col = np.array(data.edge_index[1])
  d = np.ones(len(data.edge_index[0]), dtype=np.float32)

  A = coo_matrix((d, (row, col)), shape=(data.num_nodes, data.num_nodes))

  np.array(A.sum(axis=0))[0]
  D = diags(np.array(A.sum(axis=0))[0], 0)
  A_t = A + diags(np.ones(A.shape[0]) - A.diagonal())
  D_t = diags(np.array(A_t.sum(axis=0))[0], 0)

  D_t_power = D_t.power(-0.5)
  D_t_power.toarray()
  
  A_norm = coo_matrix(A_t.dot(D_t_power).transpose().dot(D_t_power))
  return A_norm


#### Question 14 (1 pt)

To prepare the batch of graphs, we need to collect multiple adjacency matrices, feature matrices, and labels.

When using the trick described in the last figure, we see that we have to keep track of when a graph starts and when it ends, so that we can later differentiate the outputs due to $X^{(0)}$, $X^{(1)}$, etc. To achieve this, we will additionally collect a 1D array of batch indices, one for each $X^{(i)}$.

The 1D array has as many elements as rows in $X^{(i)}$, and it is filled with the value $i$ (the position of $X^{(i)}$ in the batch):

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/03-batch-indices.png">

We will later concatenate all the 1D arrays along the first dimension, just as we will do with all the $X^{(i)}$.

Define a function `prepare_graphs_batch()` that takes as input a dataset (e.g. `train_dataset`), and does the following

- Create four empty lists:
  - `adj_matrices`
  - `feature_matrices`
  - `batch_indices`
  - `labels`
- Iterate over the input dataset, getting one graph at a time. At each step, use your function from Question 13 to append the adjacency matrix to `adj_matrices`, append the matrix of input features to `feature_matrices`, create the array of batch indices (as explained above) and append it to `batch_indices`, and append the label of the graph to `labels`. **Make sure to convert the label to float**.
- Once the loop is over, use `scipy.sparse.block_diag()` to build the block diagonal matrix $\hat{A}_B$. Convert it to the COO format, and then use your answer to Question 6 to turn it into a sparse PyTorch tensor.
- Use `torch.cat()` to concatenate the tensors in `feature_matrices` along the first dimension. Do this also for `batch_indices` and `labels`.
- Return the 4 tensors computed in the previous two items.

In [ ]:
# Your answer here
def prepare_graphs_batch(dataset):
  adj_matrices = []
  feature_matrices = []
  batch_indices = []
  labels = []

  for idx, i in enumerate(dataset):
    adj_matrices.append(get_a_norm(i))
    feature_matrices.append(i.x)
    batch = torch.from_numpy(np.full((i.num_nodes), idx)) 
    batch_indices.append(batch)
    labels.append(i.y[:])

  A_hat_b = coo_matrix(block_diag(adj_matrices))

  A_torch = to_torch(A_hat_b)

  feature = torch.cat(feature_matrices, 0).requires_grad_(True)

  batch = torch.cat(batch_indices, 0)

  label = torch.cat(labels,0).to(torch.float)

  return A_torch, feature, batch, label

Once your answer for the previous question is ready, you can run the next cell to prepare all the required information, for the train, validation, and test splits.

In [ ]:
train_a_norm, train_features, train_batch_idx, train_labels = prepare_graphs_batch(train_dataset)
valid_a_norm, valid_features, valid_batch_idx, valid_labels = prepare_graphs_batch(valid_dataset)
test_a_norm, test_features, test_batch_idx, test_labels = prepare_graphs_batch(test_dataset)

### GCNs for graph classification

We now have all the ingredients to pass a batch of graphs to a GCN. However, for each graph in the batch, the output $Z^{(i)}$ contains one row for each node in the graph. If the goal is to do classification at the graph level, we have to *pool* these vectors to then compute the required logits for classification.

This operation is similar as how pooling works in a CNN. We could consider taking the mean of the vectors, the sum, or use max-pooling. The difference with respect to CNNs is that in our case, we have a batch of graphs, each potentially with a different number of nodes.

To implement this specific pooling, we can use the scatter operation in the `torch_scatter` library, which comes when installing PyG. We will use it, together with the tensor of batch indices from the previous two questions, to pool the outputs of the GCN for each graph, into a single vector:

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/04-scatter.png">

You can check more details in the [documentation](https://pytorch-scatter.readthedocs.io/en/latest/functions/scatter.html).

#### Question 15 (1.0 pt)

Implement a `GraphClassifier` module using PyTorch.

- The constructor should take as arguments the number of input features, the hidden dimension, and the number of classes.
- The model should contain a instance of the `GCN` module (as you implemented it in Question 8). Use the same value for the hidden dimension and the number of output features (recall that your `GCN` module from Question 8 has two GCN layers).
- The model should also contain a `torch.nn.Linear` layer, with the hidden dimension as the input features, and the number of classes as the output.
- The forward method receives the concatenated matrix of features, the sparse block diagonal adjacency matrix, and the batch indices (the latter is used when calling `scatter`).
- Use the following architecture in the forward pass:
  - GCN $\to$ ReLU $\to$ scatter (max) $\to$ Linear.

The output of the forward should be a 1D tensor (you might need to call `squeeze` to get rid of extra dimensions) containing the logits for all graphs in the batch, for the binary classification task.

In [ ]:
# Your answer here
import scipy

class GraphClassifier(nn.Module):
  """
  Inits a weight matrix
  Inputs:
    c_in - Number of input features
    hid - hidden dimensions
    c_out - Number of output features
  """
  def __init__(self, c_in, c_hidden, c_out):
      super(GraphClassifier, self).__init__()
      self.layer1 = GCNLayer(c_in, c_hidden) 
      self.layer2 = nn.Linear(c_hidden, c_out)
      

  def forward(self, X, A_hat, batch_indices):
      """
      Inputs:
          X - Tensor of shape [#nodes, c_in] or [#nodes, c_in]
          A_hat - Adjacency matrices of the graph. 
          batch_indices - Indices that locates the different batches
      """
      x = self.layer1(X, A_hat).relu()
      out, _ = scatter_max(x, batch_indices, dim=0)
      layer2_out = self.layer2(out)
      output = torch.sigmoid(torch.squeeze(layer2_out))
      return output

#### Question 16 (1.5 pt)

Implement a training loop for the graph classifier. Use the data from Question 14 to train and evaluate the model.

We encourage you to use a GPU in this section for faster training. Note that if you change the runtime at this point, you must re-execute several of the cells above, including the ones that install PyG.

- Instantiate a classifier with 32 as the hidden dimension
- Use Adam with a learning rate of 1e-3.
- Use `torch.nn.BCEWithLogitsLoss` as the loss function.
- Train for 5,000 epochs. Once training is done, plot the loss curve and the accuracy in the validation set. Then report the accuracy in the test set.

**Note:** the logits from the output of the classifier come from a linear layer. To compute actual predictions for the calculation of the accuracy, pass the logits through `torch.sigmoid()`, and set the predicted values to 1 whenever they are greater than 0.5, and to 0 otherwise.

You should get an accuracy equal to or higher than 70% in the validation and test sets. Can you beat the [state-of-the-art](https://paperswithcode.com/sota/graph-classification-on-proteins)? Feel free to modify your architecture and experiment with it.

Discuss what you observe during training and your results.

In [ ]:
# If your runtime is GPU-enabled, use .to(device) to move the model
# and all the relevant tensors to the GPU. You have to move tensors back to CPU
# when computing metrics like accuracy, using .cpu().
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'


# Your answer here
def forward_Graph(features, model, loss_BCE, optim, A_hat, batch_idx, Y, mode="train"):
  # Compute the model prediction
  pred = model(features, A_hat, batch_idx)

  #compute Loss & accuracy 
  running_loss = loss_BCE(pred, Y)
  correct = ((pred > .5).float() == Y)
  accuracy = correct.sum() / len(Y)

  # Training step
  if mode == "train":
    optim.zero_grad()
    running_loss.backward()
    optim.step()

  return running_loss, accuracy

# Check for availability of device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create model
model2 = GraphClassifier(c_in=4, c_hidden=32, c_out=1)
model2.to(device)
model2.train()

# Send to the available devide
train_a_norm= train_a_norm.to(device) 
train_features= train_features.to(device)
train_batch_idx = train_batch_idx.to(device) 
train_labels = train_labels.to(device)
valid_a_norm = valid_a_norm.to(device) 
valid_features = valid_features.to(device) 
valid_batch_idx = valid_batch_idx.to(device)
valid_labels = valid_labels.to(device)
test_a_norm = test_a_norm.to(device)
test_features = test_features.to(device)
test_batch_idx = test_batch_idx.to(device)
test_labels = test_labels.to(device)

# Set hyperparameters
learning_rate = 1e-3
epochs = 5000

# Set optimizer & loss function
optimizer = torch.optim.Adam(model2.parameters(), lr = learning_rate, weight_decay=5e-4)
loss_BCE = nn.BCEWithLogitsLoss()

# Training loop 
for e in range(epochs):
  running_loss, accuracy = forward_Graph(train_features, model2, loss_BCE, optimizer, train_a_norm, train_batch_idx, train_labels)
  if e % 100 == 0:
    print("Epoch number", e+1, "Loss:", running_loss.item(), "Acc", accuracy.item())

# Model evaluation
print("---Validation---")
loss, accuracy = forward_Graph(test_features, model2, loss_BCE, optimizer, test_a_norm, test_batch_idx, test_labels, mode="test")
print("Loss:", loss.item(), "Acc", accuracy.item())

print("---Test---")
loss, accuracy = forward_Graph(valid_features, model2, loss_BCE, optimizer, valid_a_norm, valid_batch_idx, valid_labels, mode="validation")
print("Loss:", loss.item(), "Acc", accuracy.item())


Epoch number 1 Loss: 0.8367689251899719 Acc 0.4074074327945709
Epoch number 101 Loss: 0.7044797539710999 Acc 0.5925925970077515
Epoch number 201 Loss: 0.693560779094696 Acc 0.5925925970077515
Epoch number 301 Loss: 0.6917855739593506 Acc 0.5925925970077515
Epoch number 401 Loss: 0.6911154389381409 Acc 0.5925925970077515
Epoch number 501 Loss: 0.690687894821167 Acc 0.5925925970077515
Epoch number 601 Loss: 0.6903194189071655 Acc 0.5925925970077515
Epoch number 701 Loss: 0.689959704875946 Acc 0.5925925970077515
Epoch number 801 Loss: 0.689557671546936 Acc 0.5925925970077515
Epoch number 901 Loss: 0.6890342235565186 Acc 0.5925925970077515
Epoch number 1001 Loss: 0.6882314085960388 Acc 0.5925925970077515
Epoch number 1101 Loss: 0.687152087688446 Acc 0.5925925970077515
Epoch number 1201 Loss: 0.6858388185501099 Acc 0.5948373079299927
Epoch number 1301 Loss: 0.6842437386512756 Acc 0.6127946376800537
Epoch number 1401 Loss: 0.6822535991668701 Acc 0.6240180134773254
Epoch number 1501 Loss: 0.6

Finally we conclude with a small summary of the observations we found during training. In the first epoch we expect the accuracy to be close to the .50, because at every single observation we have a 50% chance of guessing correctly. This also holds when running the model. Next we see that we train on the data and we see that the model trains and quite quickly (approx. after 100 epochs) the accuracy stabalizes. Even after 5000 epochs the accuracy appears to be "stuck" in a local minimum. We clearly see that the model does not over fit even though it sees exactly the same data at every epoch. This can be found especially well when comparing the train accuracy, with that of the test and validation set.

## The end

If you have made it all the way here successfully, congratulations! 🎉 

You have implemented your own GCN and tested it on a node classification task, and a more challenging classification task over multiple graphs.

We hope you can use this knowledge to apply GCNs not only to the tasks described here, but other applications where data can be modeled as a graph.

If you are interested in applying graph neural networks to larger graphs, or try newer architectures, you can dive deeper into [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/), a library with fast implementations for a wide range of architectures. It also comes with custom code that takes care of aspects that you dealt with manually for this assignment, like a more efficient implementation of the adjacency matrix multiplication via message-passing methods, and Data Loaders that relieve you from having to build block diagonal sparse matrices.

You can also check the [Deep Graph Library](https://docs.dgl.ai/) another powerful library for deep learning on graphs which also integrates with other backends like TensorFlow.

# Grading (10pt)

- Question 1: 0.25pt 
- Question 2: 0.25pt 
- Question 3: 0.5pt 
- Question 4: 0.25pt 
- Question 5: 0.5pt 
- Question 6: 0.5pt 
- Question 7: 0.5pt 
- Question 8: 0.5pt 
- Question 9: 1.5pt 
- Question 10: 0.5pt 
- Question 11: 0.25pt 
- Question 12: 0.5pt 
- Question 13: 0.5pt 
- Question 14: 1pt
- Question 15: 1pt
- Question 16: 1.5pt